## Wikidata Knowledge Graph Extraction
Many recommendation algorithms (DKN, RippleNet, KGCN) use Knowledge Graphs as an external source of information. We found that one of the bottlenecks to benchmark current algorithms like DKN, RippleNet or KGCN is that they used Microsoft Satori. As Satori is not open source, it's not possible to replicate the results found in the papers. The solution is using other open source KGs.

The goal of this notebook is to provide examples of how to interact with Wikipedia queries and Wikidata to extract a Knowledge Graph that can be used with the mentioned algorithms.

The steps covered are:
- How to find a Wikidata entity (https://www.wikidata.org/wiki/Wikidata:Glossary/en from a text query
- How to find surrounding entities of an entity 
- How to find the description of an entity
- Create a KG for Movielens

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
print("System version: {}".format(sys.version))

import pandas as pd
from reco_utils.dataset.wikidata import (
    find_wikidataID,
    query_entity_links,
    read_linked_entities,
    query_entity_description
)

import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

from reco_utils.dataset import movielens
from reco_utils.common.notebook_utils import is_jupyter

System version: 3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 09:55:02) 
[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]


In [2]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'
MOVIELENS_SAMPLE = False
MOVIELENS_SAMPLE_SIZE = 5

In [3]:
KG_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + '_wikidata.csv'

## 1. Create a KG from linked entities in Wikidata

In [4]:
names = ["The Godfather", "Al Pacino", "Tom Hanks", "Forrest Gump", "Julia Roberts", "", "My Best Friend's Wedding"]

In [7]:
def wikidata_KG_from_list(names):
    results_list = pd.DataFrame()
    for n in names:
        print(n)
        entity_id = find_wikidataID(n)
        if entity_id != "entityNotFound":
            json_links = query_entity_links(entity_id)
            related_entities,related_names = read_linked_entities(json_links)
            d = pd.DataFrame({
                "name":n,
                "original_entity":[entity_id]* len(related_entities),
                "linked_entities":related_entities,
                "name_linked_entities":related_names})
            results_list = pd.concat([results_list, d])
    return results_list

In [8]:
%%time
results_list = wikidata_KG_from_list(names)
results_list.head()

The Godfather
Al Pacino
Tom Hanks
Forrest Gump
Julia Roberts

My Best Friend's Wedding
CPU times: user 450 ms, sys: 134 ms, total: 584 ms
Wall time: 31.4 s


### Visualize KG using networkx

In [9]:
G = nx.from_pandas_edgelist(results_list, 'original_entity', 'linked_entities')

In [10]:
target_names = results_list[["linked_entities", "name_linked_entities"]].drop_duplicates().rename(columns={"linked_entities": "labels", "name_linked_entities": "name"})
source_names = results_list[["original_entity", "name"]].drop_duplicates().rename(columns={"original_entity": "labels"})
names = pd.concat([target_names, source_names])
names = names.set_index("labels")
names = names.to_dict()["name"]

In [34]:
plt.figure(figsize=(12,12)) 
pos = nx.spring_layout(G)
nx.draw(G,pos, node_size=60,font_size=9, width = 0.2)
nx.draw_networkx_labels(G, pos, names, font_size=9)
plt.show()

## 2. Create an item description with short description and linked entitites

In [35]:
# Create entity description with small description and string of linked entities
names = ["The Godfather", "Al Pacino"]

In [36]:
def wikidata_descriptions_from_list(names):
    result_description = pd.DataFrame()
    for n in names:
        entity_id = find_wikidataID(n)
        if entity_id != "entityNotFound":
            json_links = query_entity_links(entity_id)
            entity_description = query_entity_description(entity_id)
            related_entities,related_names = read_linked_entities(json_links)
            d = pd.DataFrame({"name": n,
                              "original_entity": entity_id,
                              "description":entity_description,
                              "related_names":', '.join(related_names)}, index = [0])
            result_description = pd.concat([result_description, d])
    return result_description

In [37]:
%%time
result_description = wikidata_descriptions_from_list(names)
result_description.head(10)

KeyboardInterrupt: 

## 3. Create a KG from the Movielens Dataset

In [ ]:
# Obtain pairs of Movie Title - IDs from Movielens
df = movielens.load_pandas_df(MOVIELENS_DATA_SIZE,
                              ('UserId', 'ItemId', 'Rating', 'Timestamp'),
                             title_col='Title',
                             genres_col='Genres',
                             year_col='Year'
        )
movies = df[["Title", "ItemId"]].drop_duplicates().reset_index()

0.00B [00:00, ?B/s]

In [ ]:
movies["Title"][1:5]

In [ ]:
movies.shape

In [ ]:
def wikidata_KG_from_movielens(df):
    result_linked = pd.DataFrame()
    entity_id = find_wikidataID(df["Title"] + " film")
    if entity_id != "entityNotFound":
        json_links = query_entity_links(entity_id)
        related_entities,related_names = read_linked_entities(json_links)
        d = pd.DataFrame({"original_entity":[entity_id]* len(related_entities),
                          "linked_entities":related_entities,
                          "name_linked_entities":related_names,
                          "movielens_title": df["Title"],
                          "movielens_id": df["ItemId"],
                         })
        result_linked = pd.concat([result_linked, d])
    return result_linked

In [ ]:
# For notebook testing
if MOVIELENS_SAMPLE == True:
    movies = movies.sample(MOVIELENS_SAMPLE_SIZE, random_state=123)

In [ ]:
tqdm().pandas(desc="Number of movies completed")
result = pd.concat(list(movies.progress_apply(lambda x: wikidata_KG_from_movielens(x), axis=1)))

In [ ]:
result["movielens_title"].value_counts()

In [ ]:
# result.to_csv(KG_FILE_NAME, index = False)

In [ ]:
# Record results with papermill for tests - ignore this cell
if is_jupyter():
    # Record results with papermill for unit-tests
    import papermill as pm
    pm.record("lenght_result", result.shape[0])